In [1]:
import pandas as pd
from apyori import apriori

In [2]:
# 读取数据，查看数据格式
file_path = "D:\jj\py_data\datas\supermarket_data_clean.xlsx"
# file_path = "data/supermarket_data.csv"
dfs = pd.read_excel(file_path, index_col=0)

In [3]:
df = dfs[dfs["Profit"].map(lambda x: isinstance(x, (int, float)))]
df = df[df["Customer ID"].map(lambda x: isinstance(x, str))]
df = df[df["Product Name"].map(lambda x: isinstance(x, str))]
print(df)

                 Order Date  Order Date Year  Order Date Month  \
Order ID                                                         
AG-2011-2040       1/1/2011             2011                 1   
IN-2011-47883      1/1/2011             2011                 1   
HU-2011-1220       1/1/2011             2011                 1   
IT-2011-3647632    1/1/2011             2011                 1   
IN-2011-47883      1/1/2011             2011                 1   
...                     ...              ...               ...   
MX-2014-108574   31-12-2014             2014                12   
CA-2014-115427   31-12-2014             2014                12   
MX-2014-110527   31-12-2014             2014                12   
MX-2014-114783   31-12-2014             2014                12   
CA-2014-156720   31-12-2014             2014                12   

                 Order Date Day Ship Date  Ship Date Year  Ship Date Month  \
Order ID                                                       

In [4]:
# 按用户合并商品
combine_dict = df.groupby('Customer ID').apply(
    lambda x: {col: x[col].tolist()[0] if col != 'Product Name' else x[col].tolist() for col in x.columns}).to_dict()
new_data_dict = []
order_item_list = []
for key, value in combine_dict.items():
    sample = {"Customer ID": key, "Product Name": value["Product Name"]}
    new_data_dict.append(sample)
    order_item_list.append(value["Product Name"])
print(new_data_dict[0])
print(order_item_list[0])

{'Customer ID': 'AA-10315', 'Product Name': ['Fiskars Trimmer, Serrated', 'Avery Shipping Labels, Alphabetical', 'SanDisk Numeric Keypad, USB', 'Elite Shears, High Speed', 'Tenex Personal Project File with Scoop Front Design, Black', 'High Speed Automatic Electric Letter Opener', 'Polycom VVX 310 VoIP phone', 'Verbatim 25 GB 6x Blu-ray Single Layer Recordable Disc, 1/Pack', 'Acco Banker\'s Clasps, 5 3/4"-Long', 'SanDisk Memo Slips, Multicolor', 'BIC Highlighters, Blue', 'Samsung Audio Dock, Full Size', 'Nokia Speaker Phone, Full Size', 'Bush Floating Shelf Set, Metal', 'Apple Speaker Phone, with Caller ID', 'Smead Lockers, Wire Frame', 'Cardinal Binder Covers, Durable', 'Binney & Smith Pencil Sharpener, Water Color', 'Tenex Folders, Blue', 'Konica Calculator, Wireless', 'Hon File Folder Labels, Laser Printer Compatible', 'Master Caster Door Stop, Large Neon Orange', 'Staples', 'Sauder Library with Doors, Traditional', 'Motorola Signal Booster, Full Size', 'Motorola Headset, with Caller

In [5]:
# 关联分析
results = apriori(order_item_list, min_support=0.005, min_confidence=0.25)
print(results)

<generator object apriori at 0x000001B0F27C8040>


In [6]:
# 遍历结果数据
list1, list2, list3, list4 = [], [], [], []
for result in results:
    # 获取支持度,并保留3位小数
    support = round(result.support, 3)
    # 遍历ordered_statistics对象
    for rule in result.ordered_statistics:
        # 获取前件和后件并转成列表
        head_set = list(rule.items_base)
        tail_set = list(rule.items_add)
        # 跳过前件为空的数据
        if not head_set:
            continue
        # 将前件、后件拼接成关联规则的形式
        related_category = str(head_set) + '→' + str(tail_set)
        # 提取置信度，并保留3位小数
        confidence = round(rule.confidence, 3)
        # 提取提升度，并保留3位小数
        lift = round(rule.lift, 3)
        # 查看强关联规则，支持度，置信度，提升度
        print(related_category, support, confidence, lift)
        list1.append(related_category)
        list2.append(support)
        list3.append(confidence)
        list4.append(lift)

['Acco Binder Covers, Durable']→['Staples'] 0.005 0.258 2.171
['Acco Binder, Durable']→['Staples'] 0.007 0.268 2.257
['Acco Index Tab, Durable']→['Staples'] 0.008 0.351 2.956
['Acme Trimmer, Easy Grip']→['Staples'] 0.005 0.4 3.365
['Advantus Clamps, Assorted Sizes']→['Staples'] 0.005 0.333 2.804
['Advantus Door Stop, Erganomic']→['Staples'] 0.008 0.429 3.605
['Advantus Light Bulb, Black']→['Staples'] 0.005 0.348 2.926
['Advantus Stacking Tray, Erganomic']→['Staples'] 0.006 0.25 2.103
['Ames Peel and Seal, Set of 50']→['Staples'] 0.005 0.32 2.692
['Apple Signal Booster, Cordless']→['Staples'] 0.005 0.308 2.589
['Avery Binder Covers, Durable']→['Staples'] 0.006 0.281 2.366
['Avery Binder Covers, Economy']→['Staples'] 0.006 0.257 2.163
['BIC Canvas, Blue']→['Staples'] 0.006 0.29 2.442
['BIC Markers, Water Color']→['Staples'] 0.006 0.25 2.103
['BIC Pens, Fluorescent']→['Staples'] 0.005 0.286 2.404
['Binney & Smith Highlighters, Easy-Erase']→['Staples'] 0.005 0.267 2.243
['Binney & Smith Pe

In [9]:
# 保存结果
df = pd.DataFrame()
df['related_category'] = list1
df['support'] = list2
df['confidence'] = list3
df['lift'] = list4
df = df.sort_values('lift', ascending=False)
df.to_csv("D:/jj/py_data/shopping_basket_result.csv")
print('Save successfully.')

Save successfully.


In [8]:
product = 'GBC Standard Therm-A-Bind Covers'